In [1]:
import sys
import re
import itertools
import gurobipy as gp
%load_ext autoreload
%autoreload 2

# Preliminaries 

### Run these once

In [2]:
# value = "x_"+str(11)
def getSolFromFile(value, filename):
    with open(filename, "r") as fo:
        lines = fo.readlines()
    for line in lines:
        ans = re.split(" +",line) # Regular expression split. Treat consective delimiters as one
        v = ans[0]
        s = ans[1]
        if v==value:
            return( float(s))
    return 0.0

# values = ["x_11", "x_33"]
def getSolsFromFile(values, filename):
    """
    values = ["x_11", "x_33"]
    ans = getSolsFromFile(values, "dat/dbgSol.sol")
    print(ans)
    
    The above code fetches the value of variables x_11 and x_33 
    in a gurobi or a scip written solution file dat/dbgSol.sol
    and returns the values in a dictionary
    """    
    ans = {k:0.0 for k in values}
    with open(filename, "r") as fo:
        lines = fo.readlines()
    for line in lines:
        ind = re.split(" +",line) # Regular expression split. Treat consective delimiters as one
        v = ind[0]
        s = ind[1]
        if v in values:
            print(v)
            ans[v] = float(s)
    return ans
def getParLocVar(filename):
    """
    Get the set of parameters, variable locations in the lcp model and values
    from a given file written using EPEC::appendSolution4XL
    If the file has multiple solution value rows, the first of them is chosen
    """
    with open(filename, "r") as fo:
        line1 = fo.readline().split(" ")
        line2 = fo.readline().split(" ")
        line3 = fo.readline().split(" ")

    param = dict()
    var = dict()
    locs = dict()
    for w1, w2, w3 in zip(line1, line2, line3):
        if w2.strip() == '':break
        key = w1.split("_")
        if len(key)>1: key = tuple(key)
        else: key = key[0]
        if float(w2) == -1: # i.e., a Parameter
            param[key] = float(w3)
        else:
            locs[key] = int(w2)
            var[key] =  float(w3)
    return param, locs, var

def getMNElocs(locfilename, solfilename):
    """
    Get the set of variable locations in the lcp model and values
    from a given file written using EPEC::writeMNEposns
    """
    with open(locfilename, "r") as fo:
        line1 = fo.readline().split(" ")
        line2 = fo.readline().split(" ")

    locs = dict()
    var = dict()
    for w1, w2 in zip(line1, line2):
        if w2.strip() == '':break
        key = w1.split("_")
        if len(key)>1: key = tuple(key)
        else: key = key[0]
        locs[key] = int(w2)
        var[key] = getSolFromFile("x_"+ str(int(w2)) ,solfilename)
    return locs, var

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def formatIt(a,b = ""):
    if is_number(b):
        return "{:>30}: {:>8}\n".format(a,round(b,2))
    else:
        return "{:>30}: {:>8}\n".format(a,b)
    
def getGurVar(model, locationDict, variable):
    return model.getVarByName("x_"+str(locationDict[variable]))

# def getMNEvars(solutionfile, locDict):
#     for key in locDict:
        

In [3]:
class EPECio:
    def __init__(self, countries, producers, dirty, clean, n_Scen=1):
        """
        countries should be a list of countries
        producers should be a dictionary, where the keys are elements of countries. 
                producers[cc] should contain the set of producers in country cc
        dirty is the set of dirty energy technologies
        clean is the set of clean energy technologies
        """
        self.countries = countries
        self.producers = producers
        self.dirty = dirty
        self.clean = clean
        self.n_Scen = n_Scen
        self.energy = dirty+clean
    def acquireData(self,filename):
        self.param, self.locs, self.var = getParLocVar (filename)
    def writeFoll(self,filename, cc, ff):
        with open(filename,"a") as fo:
            fo.write(cc+"---"+ff+"\n")
            fo.write(formatIt("Carbon Limit",self.var[(cc,ff,"CarbBuy")]))
            fo.write("\nInvestments by "+ff+"\n")
            for gg in self.clean:
                fo.write(formatIt("Investment in "+gg,self.var[(cc,ff,"Inv",gg)]))
            fo.write("\n")
            for ee in self.energy:
                ans = self.param[(cc,ff,"Cap",gg)] + (self.var[(cc,ff,"Inv",gg)] if ee in self.green else 0)
                fo.write(formatIt("Total Capacity in "+ee, ans))
            fo.write("\n")
            # Scenario wise production
            expProd = {ee:0 for ee in self.energy}
            for xi in range(self.n_Scen):
                if self.n_Scen>1: fo.write("Scenario: "+str(xi+1)+"\n")
                if self.n_Scen>1:fo.write("  Production: "+"\n")
                for ee in self.energy:
                    val = self.var[(cc,ff,"Prod",ee,"xi"+str(xi))]
                    if self.n_Scen>1: fo.write(formatIt(ee+": ",val))
                    expProd[ee] += val/self.n_Scen
                if self.n_Scen>1: fo.write("\n")
            # Expected production
            fo.write(" EXPECTED Production: "+"\n")
            for ee in self.energy:
                fo.write(formatIt(ee+": ",expProd[ee]))
            fo.write(formatIt("Total: ",sum(expProd[ee] for ee in self.energy)))
            fo.write("\n")
            fo.write("-----------------------------------------")
            fo.write("\n")
            return expProd
                
    def writeCoun(self,filename, cc):
        with open(filename,"a") as fo:
            fo.write(cc + ":\n")
            fo.write(formatIt("Carbon Import:",self.var[(cc,"CarbImp")]))
            fo.write(formatIt("Total Emission:",self.var[(cc,"TotEmission")]))
            fo.write(formatIt("Carbon Tax:",self.var[(cc,"CarbTax")]))
        Prod = dict()
        for ff in self.producers[cc]:
            Prod[ff] = self.writeFoll(filename,cc,ff)
        with open(filename,"a") as fo:
            for ee in self.energy:
                fo.write(formatIt("Country total "+ee+": ",sum(Prod[ff][ee] for ff in self.producers[cc])))
            fo.write(formatIt("Country Total Prodn: ",sum(Prod[ff][ee] for ff in self.producers[cc]
                                                          for ee in self.energy
                                                         )))

    def writeAll(self,filename):
        with open(filename,"w") as fo:
            fo.write("###########################################\n")
            carbSuppl = sum( [self.var [(cc, "CarbImp")] for cc in self.countries ])
            fo.write(formatIt("Total Carbon Supplied:", carbSuppl))
            fo.write(formatIt("Carbon Price:", str(self.param["suppInt"]) + " + " + str(self.param["suppSlope"]) + "Q"))
            fo.write(formatIt(" ", self.param["suppInt"] + self.param["suppSlope"]*carbSuppl ))
            fo.write("###########################################\n")
        for cc in self.countries:
            self.writeCoun(filename, cc)
            with open(filename,"a") as fo:
                fo.write("###########################################\n")
                
        

# Reoptimization routines Commands

### Input the prefix

In [4]:
prefix = "suIn0"
argv = ["readWrite", "../dat/"+prefix+"solLog.dat", "../dat/"+prefix+"lcpmodel.lp", "../dat/"+prefix+"Poly_"]

In [5]:
countries = ["c1","c2"]
producers = {"c1":["F1","F2"],"c2":["F3","F4"]}
dirty = ["coal", "gas"]
clean = ["solar", "wind"]
epec = EPECio(countries, producers, dirty, clean)
epec.acquireData(argv[1])

In [6]:
epec.locs

{('c1', 'F1', 'Inv', 'solar'): 0,
 ('c1', 'F1', 'Inv', 'wind'): 1,
 ('c1', 'F1', 'Prod', 'coal', 'xi0'): 2,
 ('c1', 'F1', 'Prod', 'gas', 'xi0'): 3,
 ('c1', 'F1', 'Prod', 'solar', 'xi0'): 4,
 ('c1', 'F1', 'Prod', 'wind', 'xi0'): 5,
 ('c1', 'F1', 'CarbBuy'): 6,
 ('c1', 'F2', 'Inv', 'solar'): 7,
 ('c1', 'F2', 'Inv', 'wind'): 8,
 ('c1', 'F2', 'Prod', 'coal', 'xi0'): 9,
 ('c1', 'F2', 'Prod', 'gas', 'xi0'): 10,
 ('c1', 'F2', 'Prod', 'solar', 'xi0'): 11,
 ('c1', 'F2', 'Prod', 'wind', 'xi0'): 12,
 ('c1', 'F2', 'CarbBuy'): 13,
 ('c1', 'CarbImp'): 14,
 ('c1', 'CarbTax'): 15,
 ('c1', 'TotEmission'): 18,
 ('c2', 'F3', 'Inv', 'solar'): 188,
 ('c2', 'F3', 'Inv', 'wind'): 189,
 ('c2', 'F3', 'Prod', 'coal', 'xi0'): 190,
 ('c2', 'F3', 'Prod', 'gas', 'xi0'): 191,
 ('c2', 'F3', 'Prod', 'solar', 'xi0'): 192,
 ('c2', 'F3', 'Prod', 'wind', 'xi0'): 193,
 ('c2', 'F3', 'CarbBuy'): 194,
 ('c2', 'F4', 'Inv', 'solar'): 195,
 ('c2', 'F4', 'Inv', 'wind'): 196,
 ('c2', 'F4', 'Prod', 'coal', 'xi0'): 197,
 ('c2', 'F4'

In [7]:
M = gp.read(argv[2])
expr = M.getObjective()
expr = gp.quicksum([getGurVar(M, epec.locs, (cc,"CarbImp")) for cc in countries])
M.setObjective(expr)
# ci1 = M.getVarByName("x_14")
# ci2 = M.getVarByName("x_202")
ci1 = getGurVar(M, epec.locs, ('c1','CarbImp'))
ci2 = getGurVar(M, epec.locs, ('c2','CarbImp'))
M.addConstr(ci1==ci2)
# M.params.SolutionLimit = 1
M.params.SolFiles = "../dat/"+prefix+"dbg"
M.optimize()

Using license file /home/sriram/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file ../dat/suIn0lcpmodel.lp
Reading time = 0.01 seconds
: 1712 rows, 3424 columns, 6142 nonzeros
Changed value of parameter SolFiles to ../dat/suIn0dbg
   Prev:   Default: 
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 1713 rows, 3424 columns and 6144 nonzeros
Model fingerprint: 0x5dd639f0
Model has 1712 general constraints
Variable types: 1712 continuous, 1712 integer (1712 binary)
Coefficient statistics:
  Matrix range     [2e-08, 8e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 1e+00]
Presolve removed 1316 rows and 2895 columns
Presolve time: 0.10s
Presolved: 397 rows, 529 columns, 1333 nonzeros
Presolved model has 133 SOS constraint(s)
Variable types: 391 continuous, 138 integer (138 binary)

Root relaxation: objective 0.000000e+00, 112 iterations, 0.00 seconds

    Nodes    |    Cu

In [86]:
for sol in range(M.getAttr("SolCount")):
    newVars = dict()
    for kk in epec.locs.keys():
        vv = epec.locs[kk]
        newVars[kk] = getSolFromFile("x_" + str(vv), "../dat/" + prefix + "dbg_" + str(sol) + ".sol")
    epec.var = newVars
    epec.writeAll(argv[3]+str(sol)+".txt")
# epec.writeAll("../dat/temp2.txt")

In [65]:

newVars = dict()
for kk in epec.locs.keys():
    vv = epec.locs[kk]
    newVars[kk] = getSolFromFile("x_" + str(vv), "../dat/default1.sol")
epec.var = newVars
# epec.writeAll(argv[3]+str(sol)+".txt")
epec.writeAll("../dat/temp2.txt")

FileNotFoundError: [Errno 2] No such file or directory: '../dat/default1.sol'

In [66]:
epec.var

{('c1', 'F1', 'Inv', 'solar'): 3741.3538723489105,
 ('c1', 'F1', 'Inv', 'wind'): 1293.6644363034907,
 ('c1', 'F1', 'Prod', 'coal', 'xi0'): 0.0,
 ('c1', 'F1', 'Prod', 'gas', 'xi0'): 0.0,
 ('c1', 'F1', 'Prod', 'solar', 'xi0'): 972.303048725483,
 ('c1', 'F1', 'Prod', 'wind', 'xi0'): 599.3654238336147,
 ('c1', 'F1', 'CarbBuy'): 0.0,
 ('c1', 'F2', 'Inv', 'solar'): 3741.3538723489105,
 ('c1', 'F2', 'Inv', 'wind'): 1293.6644363034907,
 ('c1', 'F2', 'Prod', 'coal', 'xi0'): 0.0,
 ('c1', 'F2', 'Prod', 'gas', 'xi0'): 0.0,
 ('c1', 'F2', 'Prod', 'solar', 'xi0'): 972.303048725483,
 ('c1', 'F2', 'Prod', 'wind', 'xi0'): 599.3654238336147,
 ('c1', 'F2', 'CarbBuy'): 0.0,
 ('c1', 'CarbImp'): 0.0,
 ('c1', 'CarbTax'): 0.0,
 ('c1', 'TotEmission'): 0.0,
 ('c2', 'F3', 'Inv', 'solar'): 1674.5524912601545,
 ('c2', 'F3', 'Inv', 'wind'): 559.7390479168711,
 ('c2', 'F3', 'Prod', 'coal', 'xi0'): 356.81020497473565,
 ('c2', 'F3', 'Prod', 'gas', 'xi0'): 837.5736559579327,
 ('c2', 'F3', 'Prod', 'solar', 'xi0'): 465.93

In [67]:
epec.locs

{('c1', 'F1', 'Inv', 'solar'): 0,
 ('c1', 'F1', 'Inv', 'wind'): 1,
 ('c1', 'F1', 'Prod', 'coal', 'xi0'): 2,
 ('c1', 'F1', 'Prod', 'gas', 'xi0'): 3,
 ('c1', 'F1', 'Prod', 'solar', 'xi0'): 4,
 ('c1', 'F1', 'Prod', 'wind', 'xi0'): 5,
 ('c1', 'F1', 'CarbBuy'): 6,
 ('c1', 'F2', 'Inv', 'solar'): 7,
 ('c1', 'F2', 'Inv', 'wind'): 8,
 ('c1', 'F2', 'Prod', 'coal', 'xi0'): 9,
 ('c1', 'F2', 'Prod', 'gas', 'xi0'): 10,
 ('c1', 'F2', 'Prod', 'solar', 'xi0'): 11,
 ('c1', 'F2', 'Prod', 'wind', 'xi0'): 12,
 ('c1', 'F2', 'CarbBuy'): 13,
 ('c1', 'CarbImp'): 14,
 ('c1', 'CarbTax'): 15,
 ('c1', 'TotEmission'): 18,
 ('c2', 'F3', 'Inv', 'solar'): 188,
 ('c2', 'F3', 'Inv', 'wind'): 189,
 ('c2', 'F3', 'Prod', 'coal', 'xi0'): 190,
 ('c2', 'F3', 'Prod', 'gas', 'xi0'): 191,
 ('c2', 'F3', 'Prod', 'solar', 'xi0'): 192,
 ('c2', 'F3', 'Prod', 'wind', 'xi0'): 193,
 ('c2', 'F3', 'CarbBuy'): 194,
 ('c2', 'F4', 'Inv', 'solar'): 195,
 ('c2', 'F4', 'Inv', 'wind'): 196,
 ('c2', 'F4', 'Prod', 'coal', 'xi0'): 197,
 ('c2', 'F4'

# MNE

In [59]:
ll,vv = getMNElocs("../dat/default1MNE.dat", "../dat/_tempSol.sol")

In [60]:
vv

{('c1', 'nPoly'): 0.0,
 ('c1', 'poly0', 'probab'): 0.07090763571192049,
 ('c1', 'poly0', 'F1', 'Inv', 'solar'): 48.591052344156886,
 ('c1', 'poly0', 'F1', 'Inv', 'wind'): 74.82062324658287,
 ('c1', 'poly0', 'F1', 'Prod', 'coal', 'xi0'): 0.0,
 ('c1', 'poly0', 'F1', 'Prod', 'gas', 'xi0'): 60.271490355132414,
 ('c1', 'poly0', 'F1', 'Prod', 'solar', 'xi0'): 15.85233162970926,
 ('c1', 'poly0', 'F1', 'Prod', 'wind', 'xi0'): 36.61488309048515,
 ('c1', 'poly0', 'F1', 'CarbBuy'): 21.697736527847667,
 ('c1', 'poly0', 'F2', 'Inv', 'solar'): 48.59105234415704,
 ('c1', 'poly0', 'F2', 'Inv', 'wind'): 74.82062324658307,
 ('c1', 'poly0', 'F2', 'Prod', 'coal', 'xi0'): 0.0,
 ('c1', 'poly0', 'F2', 'Prod', 'gas', 'xi0'): 60.271490355132414,
 ('c1', 'poly0', 'F2', 'Prod', 'solar', 'xi0'): 15.852331629709298,
 ('c1', 'poly0', 'F2', 'Prod', 'wind', 'xi0'): 36.61488309048522,
 ('c1', 'poly0', 'F2', 'CarbBuy'): 21.697736527847667,
 ('c1', 'poly0', 'CarbImp'): 43.395473055695334,
 ('c1', 'poly0', 'CarbTax'): 5.

In [57]:
ll

{('c1', 'nPoly'): 2,
 ('c1', 'poly0', 'probab'): 186,
 ('c1', 'poly0', 'F1', 'Inv', 'solar'): 62,
 ('c1', 'poly0', 'F1', 'Inv', 'wind'): 63,
 ('c1', 'poly0', 'F1', 'Prod', 'coal', 'xi0'): 64,
 ('c1', 'poly0', 'F1', 'Prod', 'gas', 'xi0'): 65,
 ('c1', 'poly0', 'F1', 'Prod', 'solar', 'xi0'): 66,
 ('c1', 'poly0', 'F1', 'Prod', 'wind', 'xi0'): 67,
 ('c1', 'poly0', 'F1', 'CarbBuy'): 68,
 ('c1', 'poly0', 'F2', 'Inv', 'solar'): 69,
 ('c1', 'poly0', 'F2', 'Inv', 'wind'): 70,
 ('c1', 'poly0', 'F2', 'Prod', 'coal', 'xi0'): 71,
 ('c1', 'poly0', 'F2', 'Prod', 'gas', 'xi0'): 72,
 ('c1', 'poly0', 'F2', 'Prod', 'solar', 'xi0'): 73,
 ('c1', 'poly0', 'F2', 'Prod', 'wind', 'xi0'): 74,
 ('c1', 'poly0', 'F2', 'CarbBuy'): 75,
 ('c1', 'poly0', 'CarbImp'): 76,
 ('c1', 'poly0', 'CarbTax'): 77,
 ('c1', 'poly0', 'TotEmission'): 80,
 ('c1', 'poly1', 'probab'): 187,
 ('c1', 'poly1', 'F1', 'Inv', 'solar'): 124,
 ('c1', 'poly1', 'F1', 'Inv', 'wind'): 125,
 ('c1', 'poly1', 'F1', 'Prod', 'coal', 'xi0'): 126,
 ('c1', '